In [8]:
from helpers import chunk_and_contextualize_text 
import chromadb 
import ollama
chroma_client = chromadb.PersistentClient(path="chromadb/")
# chroma_client.delete_collection("mc_collection")
collection = chroma_client.create_collection(name="mc_collection", get_or_create=True)

In [9]:
count = 0
def add_to_chroma_db(text: str):
    global count 
    embed = ollama.embeddings(model='nomic-embed-text', prompt="search_document:" + text)
    collection.add(embeddings=[embed["embedding"]], documents=[text], ids=[f"id_{count}"])
    count += 1
    
def query_chroma_db(text: str, n_results=10):
    return collection.query(ollama.embeddings(model='nomic-embed-text', prompt="search_query:" + text)["embedding"], n_results=n_results)

In [10]:
%%time

# find all file in data directory 
import os
# iterate over all files in data directory
# add them into chromadb
for filename in os.listdir("data"):
    with open(f"data/{filename}", "r") as f:
        print(f"Processing {filename}")
        text = f.read()
        chunked_text = chunk_and_contextualize_text(text)
        for chunk in chunked_text:
            add_to_chroma_db(chunk)

Processing Activator_Rail.md


Processing chunks: 100%|██████████| 8/8 [00:11<00:00,  1.45s/it]


Processing Allay.md


Processing chunks: 100%|██████████| 5/5 [00:05<00:00,  1.06s/it]


Processing Allium.md


...

Processing Zombie_Villager.md


Processing chunks: 100%|██████████| 14/14 [00:15<00:00,  1.10s/it]


Processing Zombified_Piglin.md


Processing chunks: 100%|██████████| 12/12 [00:11<00:00,  1.04it/s]


CPU times: total: 33.7 s
Wall time: 54min 16s


In [13]:
def ask_rag(question, model='24k-llama3.2'):
    responses = query_chroma_db(question, n_results=10)
    # for response in responses['documents'][0]:
    #     print(response)
    context = ""
    for i in range(len(responses['documents'][0])):
        context += f"{i+1}. {responses['documents'][0][i]} "
    
    system = f"""
    You are a Minecraft expert. You are answering a question about the game of Minecraft.
    Use the following pieces of context to answer the question at the end. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Use five sentences maximum and keep the answer as concise as possible. 
    {context}
    """
    
    response = ollama.chat(model=model, messages=[
    {
        'role': 'system',
        'content': system,
    },
    {
        'role': 'user',
        'content': question,
    },
    ])
    
    return response['message']['content']

In [25]:
print(ask_rag("What does a creeper do?", model='24k-llama3.2'))

A creeper is a hostile monster mob in Minecraft that moves towards players within its detection range, making it difficult to notice until it starts hissing and preparing to explode. When close enough to a player, the creeper explodes, damaging the player and potentially destroying surrounding blocks.


In [16]:
print(ask_rag("How much health points does a Ender Dragon have?", model='24k-llama3.2'))

The Ender Dragon has 200 health points.


In [27]:
print(ask_rag("Where can I find chorus fruit?", model='24k-llama3.2'))

You can obtain Chorus Fruit by breaking Chorus Plants. Specifically, when you break one block of a Chorus Tree, there's a 50% chance (or automatically) that it will drop 0-1 Chorus Fruit.


In [29]:
print(ask_rag("What is the difference between a zombie and a husk?", model='24k-llama3.2'))

I don't know, but I can provide some information about husks. Husks are a variant of zombies that spawn in deserts, and they function similarly to zombies except they do not burn in sunlight. They also inflict Hunger with their unarmed attacks. A key difference is that husks have a higher attack strength than zombies across all difficulty levels, and they can break down doors on Hard difficulty.
